In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv('movie.csv', sep=',')
df.head

<bound method NDFrame.head of                                                     text  label
0      I grew up (b. 1965) watching and loving the Th...      0
1      When I put this movie in my DVD player, and sa...      0
2      Why do people who do not know what a particula...      0
3      Even though I have great interest in Biblical ...      0
4      Im a die hard Dads Army fan and nothing will e...      1
...                                                  ...    ...
39995  "Western Union" is something of a forgotten cl...      1
39996  This movie is an incredible piece of work. It ...      1
39997  My wife and I watched this movie because we pl...      0
39998  When I first watched Flatliners, I was amazed....      1
39999  Why would this film be so good, but only gross...      1

[40000 rows x 2 columns]>

**1. Data Pre-Processing: cleaning
text, remove stopwords,
stemming, lemmatization.**

Tahap ini melibatkan pra-pemrosesan data untuk membersihkan dan mengubah teks mentah menjadi bentuk yang dapat digunakan oleh SVM

In [3]:
## Data cleaning

text = df['text']
# 1 Lowering case: Mengubah teks menjadi huruf kecil untuk konsistensi dalam analisis
text = text.str.lower()
# text

In [4]:
# import re

# 2 Menghapus karakter khusus kecuali huruf dan angka
text = text.str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
# text

In [ ]:
# 3 Tokenisasi Memisahkan teks menjadi unit-unit yang lebih kecil seperti kata-kata atau frasa.
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

text = text.apply(lambda x: word_tokenize(x))
# text

In [ ]:
# 4 Menghapus Stopwords Menghapus kata-kata umum yang tidak memberikan informasi penting dalam analisis.
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  # Menggunakan kamus stop words Bahasa Inggris
text = text.apply(lambda x: [word for word in x if word not in stop_words])
# text

In [ ]:
# 5 Lematisasi atau Stemming: Mengubah kata-kata ke bentuk dasar mereka untuk mengurangi variasi kata yang sama
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
text = text.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
# text

In [8]:
# 6 Menghapus Kata Pendek: Menghapus kata-kata dengan panjang karakter yang terlalu pendek yang cenderung tidak memiliki makna.

min_length = 3  # Menentukan panjang minimum kata yang diizinkan
text = text.apply(lambda x: [word for word in x if len(word) >= min_length])
# text


In [9]:
# 7 Menggabungkan Kembali Teks: Menggabungkan kembali unit-unit teks yang telah dibersihkan menjadi teks yang telah diolah sepenuhnya.
text = text.apply(lambda x: ' '.join(x))
# text

**2. Feature Extraction: Bag-of-Word atau TF-IDF**

Pada tahap ini, fitur-fitur atau representasi numerik diperoleh dari teks yang telah dipreproses

In [ ]:
#Bag-of-Word
from sklearn.feature_extraction.text import CountVectorizer
new_text = text.head(15000)
# Inisialisasi CountVectorizer
vectorizer = CountVectorizer()

# Memproses teks dan membangun vocabulary
X = vectorizer.fit_transform(new_text)

# Membuat DataFrame dari hasil Bag-of-Words
df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

print(df_bow)


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

new_text = text.head(100)

# Memproses teks dan membangun TF-IDF
X = vectorizer.fit_transform(new_text)

# Membuat DataFrame dari hasil TF-IDF
df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

print(df_tfidf)

**3. Pembagian Data**

Data yang telah dipreproses dan direpresentasikan dalam bentuk fitur perlu dibagi menjadi set pelatihan (training set) dan set pengujian (test set). Set pelatihan digunakan untuk melatih model SVM, sedangkan set pengujian digunakan untuk menguji kinerja model

**4. Pelatihan Model**

Model SVM dilatih menggunakan set pelatihan yang telah dibagi sebelumnya. SVM mempelajari pola dalam data pelatihan untuk memisahkan kelas positif dan negatif. Pada tahap ini, SVM menemukan hyperplane terbaik yang memaksimalkan margin antara kelas-kelas tersebut.

**5. Evaluasi Model**

Setelah model dilatih, langkah selanjutnya adalah menguji kinerjanya pada set pengujian yang telah dipisahkan sebelumnya. Beberapa metrik evaluasi yang umum digunakan dalam analisis sentimen adalah akurasi, presisi, recall, dan F1-score. Metrik ini membantu mengukur sejauh mana model dapat mengklasifikasikan sentimen dengan benar.

**6. Prediksi**

Setelah model dievaluasi dengan baik, model tersebut dapat digunakan untuk memprediksi sentimen pada data teks baru. Teks baru akan mengalami pra-pemrosesan yang sama seperti pada tahap 1 dan diubah menjadi fitur-fitur menggunakan metode yang sama seperti pada tahap 2. Kemudian, model SVM digunakan untuk mengklasifikasikan sentimen berdasarkan fitur-fitur tersebut